In [ ]:
#load Enviornment
%run NKSOPS_02_ClassAndMethods_01.ipynb

In [ ]:
NKS_OPS_OUTLOOK_EXPORTS_INFO = NKS_OPS(input_path = r"C:\Users\76350\Documents\NKS_OPS\03_Data\01_Raw_Data\DataFiles\*.csv"
                                   , export_parent_dir = r"C:\Users\76350\Documents\NKS_OPS\03_Data\02_Processed_Data"
                                   , input_columns = None
                                   , export_columns = ['Body',"Unique_ID","From: (Name)","FILE_PATH"]
                                   , research_topic = 'NKS_OPS_Data_Study'
                                   , severity = 'H'
                                   , logfile = False
                                   , sqlite_create = r"C:\Users\76350\Documents\NKS_OPS\03_Data\03_Cleaned_Data\DataBase\NKS_OPS_DB_sqlite.db")

In [ ]:
conn = NKS_OPS_OUTLOOK_EXPORTS_INFO.create_connection(db_file = NKS_OPS_OUTLOOK_EXPORTS_INFO.sqlite_create)
cur = conn.cursor()

# Get data from sqlite
nks_ops_query = """
select * from NKS_OPS;"""

nks_flights_query = """
select * from NKS_FLIGHTS;"""

nks_ops = pd.read_sql(nks_ops_query, conn)
nks_flights = pd.read_sql(nks_flights_query, conn)

In [ ]:
# merge messages on departure airport
merged_ops_flights_deprt = nks_ops.merge(nks_flights, on="Unique_Flight_Deprt", how="left")
merged_ops_flights_deprt = merged_ops_flights_deprt[["Unique_ID", "Received","MESSAGETYPE","Unique_Flight_Deprt"
                                         ,"Unique_Flight_Arrvl_y","TAIL","Event_Order","Unique_Flight","Actual_Delay","Delay_Code"]]

# merge messages on arrival airport
merged_ops_flights_arrvl = nks_ops.merge(nks_flights, on="Unique_Flight_Arrvl", how="left")
merged_ops_flights_arrvl = merged_ops_flights_arrvl[["Unique_ID", "Received","MESSAGETYPE","Unique_Flight_Deprt_y"
                                         ,"Unique_Flight_Arrvl","TAIL","Event_Order","Unique_Flight","Actual_Delay","Delay_Code"]]

# merge messages on depature and arrival airport
merged_ops_flights_deprt_arrvl = merged_ops_flights_deprt.merge(merged_ops_flights_arrvl[['Unique_ID','Unique_Flight_Arrvl','Unique_Flight','Actual_Delay','Delay_Code']], on="Unique_ID", how="inner", suffixes=('_Dep','_Arr'))
merged_ops_flights_deprt_arrvl = merged_ops_flights_deprt_arrvl.drop(columns=['Unique_Flight_Arrvl_y'])
move_col = merged_ops_flights_deprt_arrvl.columns.tolist()
move_col.insert(5, move_col.pop(move_col.index('Unique_Flight_Deprt')))
merged_ops_flights_deprt_arrvl = merged_ops_flights_deprt_arrvl.reindex(columns= move_col)
merged_ops_flights_deprt_arrvl.to_csv(r"C:\Users\76350\Documents\NKS_OPS\03_Data\03_Cleaned_Data\DataFiles\merged_ops_flights_deprt_arrvl.csv")

# display(merged_ops_flights_deprt_arrvl[(merged_ops_flights_deprt_arrvl['TAIL'] =="527")])
display(merged_ops_flights_deprt_arrvl.sort_values(by=('Received'), ascending= False).head())


In [ ]:
# get description frequency
nks_ops_ca = nks_ops[nks_ops['MESSAGETYPE'] == "MX AIRCRAFT OOS RETURN TO SERVICE MESSAGE"]
word_dist = nltk.FreqDist(nks_ops_ca['DESCRIPTION'])
word_dist_df = pd.DataFrame(word_dist.most_common(100),columns=['Word','Frequency'])
display(word_dist_df)


In [ ]:
# nks_ops_tokenize['tokenized_sents'] = nks_ops.apply(lambda row: nltk.word_tokenize(row['DESCRIPTION']), axis=1)
# display(nks_ops_tokenize)

nks_ops_tokenize = nks_ops.DESCRIPTION.str.split(expand=True).stack().value_counts()[:20]
display(pd.DataFrame(nks_ops_tokenize).head())

# tokenize the sentences into words
nks_ops['words'] = nks_ops.DESCRIPTION.str.strip().str.split('[\W_]+')
nks_ops_word = nks_ops[['TAIL','DESCRIPTION','words']]

# list each word as a row
rows = list()
for row in nks_ops_word[['TAIL', 'words']].iterrows():
    r = row[1]
    for word in r.words:
        rows.append((r.TAIL, word))

words = pd.DataFrame(rows, columns=['TAIL', 'word'])

# remove empty strings
words = words[words.word.str.len() > 0]

# get word counts per tail
counts = words.groupby('TAIL')\
    .word.value_counts()\
    .to_frame()\
    .rename(columns={'word':'n_w'})

# function to get top_n and plot
def pretty_plot_top_n(series, top_n=5, index_level=0):
    r = series\
    .groupby(level=index_level)\
    .nlargest(top_n)\
    .reset_index(level=index_level, drop=True)
#     r.plot.bar()
    return r.to_frame()


pretty_plot_top_n(counts['n_w'])




In [ ]:
ac_damage = merged_ops_flights_deprt_arrvl[merged_ops_flights_deprt_arrvl['Event_Order'].str.contains("BIRD|DAMAGE|STRIKE|DMG|LIGHTNNING", na=False)]
display(ac_damage.sort_values(by=('Received'), ascending= False))

In [ ]:
# new_df = nks_ops[["Unique_ID","TAIL"]]
# new_df= new_df.groupby(by=["TAIL"]).count().reset_index().sort_values("Unique_ID", ascending =False).head(20)
# display(new_df)

# import matplotlib.pyplot as plt 
# new_df.plot(kind='bar',x='TAIL',y='Unique_ID') 
# plt.show() 

In [ ]:
conn.close()